### H-Pile Design 
#### Per NHI Courses No. 132021 and 132022 "Design and Construction of Driven Pile Foundations - Volume 1 & 2"
####     "LRFD Bridge Design Specifications" 9th Edition 2020 

In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import Button
from IPython.display import display
from ipyfilechooser import FileChooser
import io
import json

c:\Users\millern8989\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Pile Structural Information

In [20]:
#load csv with all the H-Pile info
H_Pile = pd.read_csv("https://raw.githubusercontent.com/geotechnick/Geotechnical_Design/main/Pile_Design/H_Piles/HP_Spec_Table.csv")
Fy = 50 # Pile yield stress in ksi
pile_names = H_Pile['Section_E'].unique().tolist()

#filter out the metric values
H_Pile = H_Pile.loc[:, ~H_Pile.columns.str.endswith('_M')]

#create drop down with the pile names 
dropdown = widgets.Dropdown(options=pile_names, description='Select Value:')

# Function to display the selected row 
def display_row(selected_value):
    global Pile_Info
    Pile_Info = H_Pile[H_Pile['Section_E'] == selected_value] #creates dataframe that only has the selected value in it
    pile_units = H_Pile[H_Pile['Section_E'] == 'Units_English'] #create separate dataframe that has units
    pile_display = pd.concat([Pile_Info, pile_units], ignore_index=True) #combine pile properties and pile units into single dataframe
    
    ## adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = Pile_Info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    Pile_Info = Pile_Info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    Pile_Info.reset_index(inplace=True)

    display(pile_display.T)
# Connect the dropdown widget to the display function using interact
widgets.interact(display_row, selected_value=dropdown) 

interactive(children=(Dropdown(description='Select Value:', options=('Units_English', 'HP 8x36', 'HP 10x42', '…

<function __main__.display_row(selected_value)>

#### Upload Soil Profile

In [21]:
# Function to handle file upload
def handle_upload(change):
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = io.BytesIO(content)
    df = pd.read_csv(content_bytes)

    # For now, let's just print the first few rows
    print(df.head())
    # Save the dataframe as a global variable so it can be used outside of the function 
    global Layer_Properties
    Layer_Properties = df

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.csv')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

FileUpload(value=(), accept='.csv', description='Upload')

           layer  lv_top_elev  mlv_top_elev  hv_top_elev  lv_bottom_elev  \
0        Layer I         10.5          10.5         10.5              -1   
1  Layer II Clay         -1.0           0.0          1.0              -6   
2      Layer III         -6.0          -5.0         -4.0             -32   
3  Layer IV Clay        -32.0         -31.0        -30.0             -47   
4        Layer V        -47.0         -46.0        -45.0             -59   

   mlv_bottom_elev  hv_bottom_elev  lv_top_depth  mlv_top_depth  hv_top_depth  \
0                0               1           NaN            NaN           NaN   
1               -5              -4           NaN            NaN           NaN   
2              -31             -30           NaN            NaN           NaN   
3              -46             -45           NaN            NaN           NaN   
4              -58             -57           NaN            NaN           NaN   

   ...  lv_residual_coh  mlv_residual_coh  hv_residual_c

In [22]:
#Should add Pile_Axial_Info with ground_surface, top_pile, pile_length, predrill_depth, steel_strength
# Define the widgets for user input
ground_surface_text = widgets.Text(description="Ground Surface:")
top_pile_text = widgets.Text(description="Top Pile:")
pile_length_float = widgets.FloatText(description="Pile Length:")
predrill_depth_float = widgets.FloatText(description="Predrill Depth:")
steel_strength_float = widgets.FloatText(description="Steel Strength:")

# Function to handle user input and create DataFrame
def create_dataframe(ground_surface, top_pile, pile_length, predrill_depth, steel_strength):
    global Pile_Axial_Info
    data = {
        "Ground Surface": [ground_surface],
        "Top Pile": [top_pile],
        "Pile Length": [pile_length],
        "Predrill Depth": [predrill_depth],
        "Steel Strength": [steel_strength]
    }
    Pile_Axial_Info = pd.DataFrame(data)
    
    ## adds case_1 to the data frame
    # Get the current index label of the first row
    old_index_label = Pile_Axial_Info.index[0]

    # Define the new label for the first row
    new_index_label = 'case_1'

    # Rename the first row
    Pile_Axial_Info = Pile_Axial_Info.rename(index={old_index_label: new_index_label})

    # Now the first row is renamed and shifted
    Pile_Axial_Info.reset_index(inplace=True)

    return Pile_Axial_Info

# Button to trigger dataframe creation
submit_button = widgets.Button(description="Submit")

def on_submit_button_clicked(b):
    display(create_dataframe(ground_surface_text.value, top_pile_text.value, pile_length_float.value, predrill_depth_float.value, steel_strength_float.value))

submit_button.on_click(on_submit_button_clicked)

# Display the widgets and button
input_widgets = widgets.VBox([ground_surface_text, top_pile_text, pile_length_float, predrill_depth_float, steel_strength_float, submit_button])
display(input_widgets)

,index,Ground Surface,Top Pile,Pile Length,Predrill Depth,Steel Strength
0,case_1,8,9,90.0,5.0,50.0


### Save properties to JSON

In [23]:
def save_json_to_file(json_data, file_path):
    if not file_path.endswith('.json'):
        file_path += '.json'
    with open(file_path, 'w') as json_file:
        json_file.write(json_data)
    print(f"JSON data saved to {file_path}")

def save_json_widget(json_data):
    file_chooser = FileChooser()
    file_chooser.use_dir_icons = True
    file_chooser.title = "Select a location to save the JSON file"
    save_button = widgets.Button(description="Save JSON")

    def save_json(button):
        if file_chooser.selected:
            save_json_to_file(json_data, file_chooser.selected)
        else:
            print("Please select a valid file path.")

    save_button.on_click(save_json)
    return widgets.VBox([file_chooser, save_button])

# Create JSON File ###############################
# Convert DataFrames to dictionaries
dict1 = Pile_Info.to_dict(orient='records')
dict2 = Layer_Properties.to_dict(orient='records')
dict3 = Pile_Axial_Info.to_dict(orient='records')

# Create a dictionary with dictionaries
pile_dict = {'Pile_Info': dict1, 'Layer_Properties': dict2, 'Pile_Axial_Info': dict3}

# Convert the dictionary to JSON format
json_data = json.dumps(pile_dict, indent=4)
###################################################

# Display the widget
save_json_widget(json_data)

JSON data saved to C:\Users\millern8989\OneDrive - ARCADIS\Documents\Python Calculations\test_save.json


### Upload from save file


In [6]:
def handle_upload(change):
    global Pile_Info
    global Layer_Properties
    global Pile_Axial_Info
    uploaded_file = change['new'][0]
    content = uploaded_file['content']
    content_bytes = content.tobytes()
    content_str = content_bytes.decode('utf-8')
    data = json.loads(content_str)

    # Check if 'Pile_Info' key exists and create a DataFrame
    if 'Pile_Info' in data:
        Pile_Info = pd.DataFrame(data['Pile_Info'])
        print("DataFrame for 'Pile_Info':")
        print(Pile_Info)
        # Optionally, you can save the DataFrame to a file
        # pile_info_df.to_csv("pile_info_data.csv", index=False)
    else:
        print("The key 'Pile_Info' does not exist in the JSON data.")

    # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Layer_Properties' in data:
        Layer_Properties = pd.DataFrame(data['Layer_Properties'])
        print("\nDataFrame for 'Layer_Properties':")
        print(Layer_Properties)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Layer_Properties' does not exist in the JSON data.")

        # Check if 'Soil_Properties' key exists and create a DataFrame
    if 'Pile_Axial_Info' in data:
        Pile_Axial_Info = pd.DataFrame(data['Pile_Axial_Info'])
        print("\nDataFrame for 'Pile_Axial_Info':")
        print(Pile_Axial_Info)
        # Optionally, you can save the DataFrame to a file
        # soil_properties_df.to_csv("soil_properties_data.csv", index=False)
    else:
        print("The key 'Pile_Axial_Info' does not exist in the JSON data.")

# Creating file upload widget
file_upload = widgets.FileUpload(accept='.json')

# Attaching the event handler
file_upload.observe(handle_upload, names='value')

# Displaying the widget
display(file_upload)

FileUpload(value=(), accept='.json', description='Upload')

DataFrame for 'Pile_Info':
  Section_E Weight_E Area_E Depth_E Flange_Width_E Flange_thick_E Web_thick_E  \
0  HP 12x53       53   15.5    11.8             12          0.435       0.435   

  Coating_area_E XX_I_E XX_S_E XX_Z_E XX_r_E YY_I_E YY_S_E YY_Z_E YY_r_E  
0           5.82    393   66.7     74   5.03    127   21.1   32.2   2.86  
The key 'Layer_Properties' does not exist in the JSON data.
The key 'Pile_Axial_Info' does not exist in the JSON data.


In [7]:
Pile_Info

,Section_E,Weight_E,Area_E,Depth_E,Flange_Width_E,Flange_thick_E,Web_thick_E,Coating_area_E,XX_I_E,XX_S_E,XX_Z_E,XX_r_E,YY_I_E,YY_S_E,YY_Z_E,YY_r_E
0,HP 12x53,53,15.5,11.8,12,0.435,0.435,5.82,393,66.7,74,5.03,127,21.1,32.2,2.86


In [8]:
Layer_Properties

,layer,lv_top_elev,mlv_top_elev,hv_top_elev,lv_bottom_elev,mlv_bottom_elev,hv_bottom_elev,lv_top_depth,mlv_top_depth,hv_top_depth,...,lv_residual_coh,mlv_residual_coh,hv_residual_coh,lv_effective_phi,mlv_effective_phi,hv_effective_phi,lv_effective_coh,mlv_effective_coh,hv_effective_coh,Unnamed: 34
0,Layer I,10.5,10.5,10.5,-1,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Layer II Clay,-1.0,0.0,1.0,-6,-5,-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Layer III,-6.0,-5.0,-4.0,-32,-31,-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Layer IV Clay,-32.0,-31.0,-30.0,-47,-46,-45,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Layer V,-47.0,-46.0,-45.0,-59,-58,-57,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Layer VI dense sand,-59.0,-58.0,-57.0,-63,-62,-61,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Layer VII Clay,-63.0,-62.0,-61.0,-79,-78,-77,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Layer VIII,-79.0,-78.0,-77.0,-124,-123,-122,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Layer IX,-124.0,-123.0,-122.0,-161,-160,-159,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
Pile_Axial_Info

,index,Ground Surface,Top Pile,Pile Length,Predrill Depth,Steel Strength
0,case_1,8,9,90.0,5.0,50.0


In [ ]:
class pile_design:
    
    def __init__(self):
        self.Pile_Info = Pile_Info
        self.Layer_Properties = Layer_Properties

    def pile_length(self,length):
        self.length = length
        